# Prediction - regression and classification

Line:

%matplotlib inline

allows you to display charts without using plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Linear Regression
This method assumes that the relationship between the explained and the explanatory variable is linear, i.e. we are trying to find the function:

y = a * x + b

that best describes the points from the set, where:

- y - explained variable,
- x - explanatory variable,
- a - slope coefficient,
- b - intercept.

For the regression example, we will use generated points randomly spaced from the line with slope coefficient a = 2 and intercept b = -3.

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = 2 * x - 3 + rng.randn(50)
plt.scatter(x, y);

We import the model for linear regression from the scikit-learn package (sklearn) and create the appropriate object. When creating an object, we can set values for the model's hyperparameters. One of such parameters is *fit_intercept*, which indicates whether the model should determine the value for the intercept **b**.

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
model

The explanatory data should have dimension $<$number of samples$>$ x $<$number of dimensions/features$>$, so we need to change the dimensions for **x**.

In [ ]:
X = x[:, np.newaxis]
X.shape

We train the model on the data. We can view the determined values for model parameters **a** and **b**.

In [ ]:
model.fit(X, y)
# a
model.coef_ 

In [ ]:
#b
model.intercept_ 

And the accuracy of the model.

In [ ]:
model.score(X, y)

Applying the designated linear regression model to "new" data.

In [ ]:
xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis])

# Visualization of results
plt.scatter(x, y)
plt.plot(xfit, yfit,'-r');

### Task 1. (0.5 points)
> Using a phone company's data and a linear regression model, determine an exact per-minute rate.

In [ ]:
# your answer here...

We are looking for the relationship between body weight and brain weight for mammals in the form of a function:

y = a * x,

where *x* means brain weight and *y* means body weight.

We load the appropriate file.

In [ ]:
import pandas as pd

df = pd.read_csv('brain_body.csv', index_col='Index')
df.head()

### Task 2. (0.5 points)
> Investigate whether there is a correlation between the variables under study.

In [ ]:
# your answer here...

### Task 3. (0.5 points)
> Create a linear regression model object suitable for the dependencies in the above problem.
>
> Prepare the explanatory variable (name it *X*) and the explained variable (name it *y*) appropriately.

In [ ]:
# your answer here...

We divide the dataset into test and training sets.

In [ ]:
from sklearn.model_selection import train_test_split

X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.6)
print ('Size of training set = %d and test set = %d' % (y1.size, y2.size))

We train and evaluate the model using mean squared error.

In [ ]:
from sklearn.metrics import mean_squared_error

model.fit(X1, y1)
y2_model = model.predict(X2)

mean_squared_error(y2, y2_model)

Visualization of results.

In [ ]:
plt.scatter(X2, y2)
plt.plot(X2, y2_model,'-r');

### Task 4. (1.5 points)
> Remove outlying points from the analyzed data set and repeat the regression. Compare the results. Also compare the accuracy of the model on training and test data.

In [ ]:
# your answer here...

If the process consists of several stages, the *Pipeline* class can be used.

http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

The following example defines a process consisting of data transformation (*PolynomialFeatures*) and linear regression.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

poly_model = make_pipeline(PolynomialFeatures(7),
                           LinearRegression())

rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = np.sin(x) + 0.1 * rng.randn(50)

poly_model.fit(x[:, np.newaxis], y)
yfit = poly_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit);

### Task 5. (0.5 points)
> Test other values (smaller and larger than the example) as an argument when creating an object of the *PolynomialFeatures* class. What can you say about the model's behavior?

In [ ]:
# your answer here...

### Task 6. (1.5 points)
> For the above random data, test the regression with $L_2$ regularization and $L_1$ regularization. Draw a graph and calculate the mean squared and mean absolute errors for the three models. Compare the results.
>
> **NOTE:** Both model error measures are available in the *sklearn.metrics* module.

http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [ ]:
# your answer here...
from sklearn.linear_model import Ridge # L2
from sklearn.linear_model import Lasso # L1


### Linear regression as a function of two variables

In [ ]:
df = pd.read_csv("boston_data.csv")
df.head()

In [ ]:
X = pd.DataFrame(df.iloc[:,:-1])
y = pd.DataFrame(df.iloc[:,-1])

In [ ]:
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.8)

In [ ]:
x_train = X1[['RM', 'LSTAT']].values.reshape(-1,2)
x_test = X2[['RM', 'LSTAT']].values.reshape(-1,2)
model = LinearRegression(fit_intercept=False)
model.fit(x_train,y1)
y2_model = model.predict(x_test)
mean_squared_error(y2, y2_model)

### Task 7. (2.5 points)
> Load the *winequality-red.csv* file and divide it into training and testing sets in the ratio of 7:3. Research the correlation between available attributes and wine quality and select several (min. 2) to create a model. If necessary, use a polynomial model. Use the $L_2$ regularization and examine the mean squared and absolute mean errors as well as the coefficient of determination. Compare the results.
>
> **NOTE:** The coefficient of determination is available in the sklearn.metrics module.

In [ ]:
# your answer here...

## Support Vector Machine

In [ ]:
from sklearn import svm

We create sample data.

In [ ]:
X = np.array([[1,2],
             [5,8],
             [1.5,1.8],
             [8,8],
             [1,0.6],
             [9,11]])
y = [0,1,0,1,0,1]

We create and teach a model.

In [ ]:
clf = svm.SVC(kernel = 'linear')
clf.fit(X, y)  

In [ ]:
clf.score(X,y)

Classification of the new point.

In [ ]:
clf.predict([[0.58,0.76]])

In [ ]:
clf.predict([[10.58,10.76]])

In [ ]:
w = clf.coef_[0]
print(w)

a = -w[0] / w[1]

xx = np.linspace(0,12)
yy = a * xx - clf.intercept_[0] / w[1]

h0 = plt.plot(xx, yy, '-r')
plt.scatter(X[:, 0], X[:, 1], c = y)

Getting support vectors.

In [ ]:
clf.support_vectors_

Getting vectors indexes.

In [ ]:
clf.support_ 

Getting the number of support vectors for each class.

In [ ]:
clf.n_support_ 

### Task 8. (2.5 points)
> Perform classification using the SVM method on the *iris* dataset. Divide the data into training and test data in a 4:1 ratio. Test different values for *kernel* ('linear','rbf','poly','sigmoid') and *gamma* (0, 10, 100) for *kernel*='rbf'. Compare accuracy on training and test sets for different model parameters. Also check the *f1* values for the prediction results of the tested models.
>
> **NOTE:** The *f1* measure is available in the *sklearn.metrics* module.

Import of the *iris* dataset.

In [ ]:
from sklearn import svm

iris = datasets.load_iris()
X = iris.data[:, :2] 
y = iris.target

In [ ]:
# your answer here...